This script takes the basic neural network model to estimate tempearture developed in `NeuralNetworks/_basic_leaky_model.py` and applies it to weather data and static reservoir elements to estimate reservoir temeperature for Granby, Grand, and Shadow Mountain.

In [12]:
#high level modules
import os
import sys
import imp
import numpy as np
import pandas as pd
import pickle

# ml/ai modules
import tensorflow as tf
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
# import pydot
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.colors as colors

# custom modules
this_dir = "/Users/steeleb/Documents/GitHub/ATS-ML-Fall2023/"
imp.load_source("universals", os.path.join(this_dir, "universal_functions.py"))
from universals import load_pickle_file, predict_values_test
imp.load_source("hindcast", os.path.join(this_dir, "NeuralNetworks/_basic_leaky_implementation_updated.py"))
from hindcast import test_ensemble, test_ensemble_ts

11/11 [==============================] - 0s 3ms/step


2024-10-18 17:10:35.059254: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-10-18 17:10:35.155241: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-10-18 17:10:35.248661: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


11/11 [==============================] - 0s 3ms/step


2024-10-18 17:10:35.348982: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-10-18 17:10:35.441551: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-10-18 17:10:35.535973: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


11/11 [==============================] - 0s 3ms/step


...

2024-10-18 17:10:35.841690: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


11/11 [==============================] - 0s 3ms/step


2024-10-18 17:10:36.236810: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-10-18 17:10:36.336199: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-10-18 17:10:36.428311: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


11/11 [==============================] - 0s 3ms/step


2024-10-18 17:10:36.522572: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
/Users/steeleb/Documents/GitHub/ATS-ML-Fall2023/NeuralNetworks/_basic_leaky_implementation_updated.py:120: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_selected["LOO_ensemble_pred"] = np.mean(test_selected[['p_act_1', 'p_act_2', 'p_act_3', 'p_act_4', 'p_act_5', 'p_act_6', 'p_act_7', 'p_act_8', 'p_act_9']], axis=1)
/Users/steeleb/Documents/GitHub/ATS-ML-Fall2023/NeuralNetworks/_basic_leaky_implementation_updated.py:121: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the 

In [13]:
test_ensemble.to_csv(os.path.join("~/Documents/GitHub/NASA-NW/data/NN_train_val_test/regional_daily_temp/", "updated_test_ensemble.csv"))
test_ensemble_ts.to_csv(os.path.join("~/Documents/GitHub/NASA-NW/data/NN_train_val_test/regional_daily_temp/", "updated_test_ensemble_ts.csv"))

First, we'll load the LOO models that we saved from the `NeuralNetworks/_basic_leaky_model.py` script.

In [15]:
# model directory path
model_dir = '/Users/steeleb/OneDrive - Colostate/NASA-Northern/data/NN_train_val_test/regional_daily_temp/models/basic_leaky_updated/'

models = [f for f in os.listdir(model_dir) if 'model' in f]

models = [f for f in models if 'ts' in f]
models.sort()

model1 = load_pickle_file(models[0], model_dir)
model2 = load_pickle_file(models[1], model_dir)
model3 = load_pickle_file(models[2], model_dir)
model4 = load_pickle_file(models[3], model_dir)


And now we'll read in the weather data and reservoir elements.

In [33]:
app_dir = "~/OneDrive - Colostate/NASA-Northern/data/NN_train_val_test/regional_daily_temp/application/"
standardized_df = pd.read_csv(os.path.join(app_dir, "application_dataset_2023_standardized.csv"))
#standardized_df = standardized_df.set_index("index")
standardized_df

df_pred = pd.read_csv(os.path.join(app_dir, "application_dataset_2023.csv"))

And apply the 6 models to the application dataset... but first we'll need to grab the mean/standard deviation values for back calculating the temperature from standardized values. 

In [26]:
nn_dir = "~/OneDrive - Colostate/NASA-Northern/data/NN_train_val_test/regional_daily_temp/"
fn = os.path.join(nn_dir, "mean_std_training_v2024-10-18.csv")
mean_std_df = pd.read_csv(fn)
mean_std_df = mean_std_df.set_index("Unnamed: 0")
mean_t = mean_std_df.loc["value"]["mean"]
std_t = mean_std_df.loc["value"]["std"]

model_1_pred = predict_values_test(model1, standardized_df, mean_t, std_t)
model_2_pred = predict_values_test(model2, standardized_df, mean_t, std_t)
model_3_pred = predict_values_test(model3, standardized_df, mean_t, std_t)
model_4_pred = predict_values_test(model4, standardized_df, mean_t, std_t)


 126/4566 [..............................] - ETA: 5s

2024-10-18 17:55:50.911036: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


 128/4566 [..............................] - ETA: 5s

2024-10-18 17:55:57.530381: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


 122/4566 [..............................] - ETA: 5s

2024-10-18 17:56:03.195457: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


 130/4566 [..............................] - ETA: 5s

2024-10-18 17:56:08.998727: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


4566/4566 [==============================] - 5s 1ms/step


In [34]:

# Create DataFrames for each model's predictions
df_model_1 = pd.DataFrame(model_1_pred, columns=['model_1'])
df_model_2 = pd.DataFrame(model_2_pred, columns=['model_2'])
df_model_3 = pd.DataFrame(model_3_pred, columns=['model_3'])
df_model_4 = pd.DataFrame(model_4_pred, columns=['model_4'])

# Concatenate the DataFrames
df_pred = pd.concat([df_pred, df_model_1, df_model_2, df_model_3, df_model_4], axis=1)

df_pred

,feature,area,shoreline_length,max_depth,date,tot_precip_mm_1,max_temp_degC_1,mean_temp_degC_1,min_temp_degC_1,tot_sol_rad_Wpm2_1,...,mean_temp_degC_10,min_temp_degC_10,tot_sol_rad_Wpm2_10,min_wind_mps_10,mean_wind_mps_10,max_wind_mps_10,model_1,model_2,model_3,model_4
0,Carter Lake,1100,12.0,180,1984-01-02,0.0000,0.49,-3.722083,-7.59,2236.252,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Carter Lake,1100,12.0,180,1984-01-03,0.0000,2.73,-3.777917,-8.35,2286.680,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Carter Lake,1100,12.0,180,1984-01-04,0.0000,6.17,2.323333,-1.94,2177.570,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Carter Lake,1100,12.0,180,1984-01-05,0.0000,7.10,3.937083,1.75,2182.700,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Carter Lake,1100,12.0,180,1984-01-06,0.0000,8.33,5.630000,4.00,2207.436,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146095,Green Mountain Reservoir,2130,19.0,524,2023-12-28,0.0000,-3.70,-9.175417,-11.63,2322.858,...,-4.775500,-16.08,20713.830,3.309038,4.767988,6.561053,5.546297,5.392278,5.470607,5.878611
146096,Green Mountain Reservoir,2130,19.0,524,2023-12-29,0.0021,-0.89,-6.348333,-10.45,2285.670,...,-5.396958,-16.08,20722.688,3.279462,4.804687,6.649800,5.765578,5.444846,5.691746,5.857488
146097,Green Mountain Reservoir,2130,19.0,524,2023-12-30,0.0008,1.25,-3.667500,-5.71,2316.980,...,-5.812750,-16.08,20803.264,3.295602,4.759350,6.487942,6.622177,5.952028,6.064677,5.927972
146098,Green Mountain Reservoir,2130,19.0,524,2023-12-31,0.0000,2.27,-3.362917,-6.12,2314.360,...,-6.134125,-16.08,20900.416,3.244472,4.840755,6.575797,7.089345,6.102879,6.118093,6.030062


And then save the file

In [35]:
df_pred.to_csv(os.path.join(app_dir, "model_predictions_2023.csv"))